In [2]:

%pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv
import sys

In [11]:
# Load environment variables from .env file
load_dotenv()

# Now you can access variables using os.getenv()
api_key = os.getenv("OPENAI_API_KEY")
print("API Key loaded successfully.")
# Initialize OpenAI API client
openai = OpenAI(api_key=api_key)

API Key loaded successfully.


In [13]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "utils")))
from create_batchFiles_openAi import generate_all_batches
question_bank = generate_all_batches()

Processing: choice_with_norm


FileNotFoundError: [Errno 2] No such file or directory: '/home/maloti12/shreshtho/self/Research/ethics/illusion_of_ethics/code/data/prompts/choice_prompts_with_norm.jsonl'